In [2]:
import pandas as pd
import bcb
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Função para obter os índices de correção entre duas datas
def obter_indices_correcao(data_inicio, data_fim, codigo_serie):
	url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=json&dataInicial={data_inicio}&dataFinal={data_fim}'

	try:
		response = requests.get(url)
		response.raise_for_status()  # Verifica se a requisição teve sucesso
		data = response.json()

		# Convertendo os dados para um DataFrame
		df = pd.DataFrame(data)
		df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
		df['valor'] = pd.to_numeric(df['valor'])
		return df

	except requests.exceptions.HTTPError as http_err:
		print(f'HTTP error occurred: {http_err}')  # Exibe o erro HTTP
	except requests.exceptions.RequestException as req_err:
		print(f'Other error occurred: {req_err}')  # Exibe outros erros de requisição
	except ValueError as val_err:
		print(f'Value error: {val_err}')  # Exibe erros de valor, como JSONDecodeError
	return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro

# Função para calcular o índice de correção
def calcular_indice_correcao(df):
	df['fator'] = 1 + df['valor'] / 100  # Converte índices percentuais em fatores
	fator_acumulado = df['fator'].prod()  # Multiplica todos os fatores
	return fator_acumulado

# Lista de datas iniciais
datas_iniciais = '01/01/2024'
data_fim = '31/01/2025'
codigo_serie = '433'  # Exemplo de código de série (Índice Nacional de Preços ao Consumidor Amplo - IPCA)

# Lista para armazenar os resultados
resultados = []

# Loop através das datas iniciais
for data_inicio in datas_iniciais:
	indices_correcao = obter_indices_correcao(data_inicio, data_fim, codigo_serie)
	if not indices_correcao.empty:
		indice_correcao = calcular_indice_correcao(indices_correcao)
		resultados.append({'data_inicial': data_inicio, 'indice_correcao': f"{indice_correcao:.10f}"})
	else:
		print(f"Não foi possível obter os índices de correção para o período começando em {data_inicio}.")

# Criar um DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)

# Exibir o DataFrame de resultados
df_resultados


HTTP error occurred: 400 Client Error: Bad Request for url: https://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=json&dataInicial=0&dataFinal=31/01/2025
Não foi possível obter os índices de correção para o período começando em 0.
Other error occurred: Expecting value: line 1 column 1 (char 0)
Não foi possível obter os índices de correção para o período começando em 1.
HTTP error occurred: 400 Client Error: Bad Request for url: https://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=json&dataInicial=/&dataFinal=31/01/2025
Não foi possível obter os índices de correção para o período começando em /.
HTTP error occurred: 400 Client Error: Bad Request for url: https://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=json&dataInicial=0&dataFinal=31/01/2025
Não foi possível obter os índices de correção para o período começando em 0.
HTTP error occurred: 400 Client Error: Bad Request for url: https://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=json&dataIni

""
